In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import json
from itertools import chain
import numpy as np

from essl.datasets import Cifar10, SVHN
from essl.backbones import largerCNN_backbone
from essl.evaluate_downstream import finetune_model
from torch.utils.data import DataLoader
import torch
import glob
import os

/home/noah/.conda/envs/essl/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_algorithm(params_path):
    with open(params_path, "r") as f:
        params = []
        for i, l in enumerate(f.readlines()):
            params.append(l)
    # get params
    ssl = [j.split(" ")[1].strip("\n") for j in params if "ssl_task" in j][0]
    bs = [j.split(" ")[1].strip("\n") for j in params if "ssl_batch_size" in j][0]
    return ssl, bs

In [3]:
# cifar10
b256_fs = glob.glob("/home/noah/ESSL/final_exps/optimization/exp8*/*")
b256 = [os.path.join(f, "outcomes.json") for f in b256_fs]
b256_names = [get_algorithm(os.path.join(f, "params.txt")) for f in b256_fs]

b32_fs = glob.glob("/home/noah/ESSL/final_exps/optimization/exp6*/*")
b32 = [os.path.join(f, "outcomes.json") for f in b32_fs]
b32_names = [get_algorithm(os.path.join(f, "params.txt")) for f in b32_fs]

# SVHN
b256_fs_svhn = glob.glob("/home/noah/ESSL/final_exps/optimization/exp10*/*")
b256_svhn = [os.path.join(f, "outcomes.json") for f in b256_fs_svhn]
b256_names_svhn = [get_algorithm(os.path.join(f, "params.txt")) for f in b256_fs_svhn]

b32_fs_svhn = glob.glob("/home/noah/ESSL/final_exps/optimization/exp11*/*")
b32_svhn = [os.path.join(f, "outcomes.json") for f in b32_fs_svhn]
b32_names_svhn = [get_algorithm(os.path.join(f, "params.txt")) for f in b32_fs_svhn]

In [4]:
def get_data(exp_path):
    chromos = []
    with open(exp_path, "r") as f:
        results = json.load(f)
        for fitness, chromo in zip(results["pop_vals"], results["chromos"]):
            c = list(chain.from_iterable(chromo[1]))
            c.append(fitness[1])
            chromos.append(c)
    columns = list(chain.from_iterable([[f"aug{i}", f"op{i}"] for i in range(1, 4)]))
    columns.append("test acc")
    columns
    df = pd.DataFrame(chromos, columns=columns)
    
    # create data in long format
    ops = set(list(df["aug1"]) + list(df["aug2"]) +  list(df["aug3"]))
    indexes = {op:i for i, op in enumerate(ops)}
    chromos_long = np.zeros([len(chromos), len(ops)+1])
    for i, c in enumerate(chromos):
        for aug, intensity in zip(c[:-2][::2], c[:-2][1::2]):
            chromos_long[i][indexes[aug]] = intensity
            chromos_long[i][-1] = c[-1]
    columns_long = list(ops) + ["fitness"]
    df_long = pd.DataFrame(chromos_long, columns = columns_long)
    return df, df_long, ops

In [5]:
for exp_path, exp_name in zip(b256, b256_names):
    df, df_long, ops = get_data(exp_path)
    print(list(df.sort_values(by="test acc").iloc[-1]))
    print(exp_name, " ", max(df_long['fitness']))

['Contrast', 0.9686110141915404, 'Solarize', 0.06499417612685054, 'VerticalFlip', 0.08094956837362144, 84.15]
('NNCLR', '256')   84.15
['Rotate', 22.0, 'TranslateY', 1.0, 'HorizontalFlip', 0.7158283743946523, 84.43]
('SimSiam', '256')   84.43
['ShearX', 0.09741248505133066, 'Brightness', 0.6905150754934473, 'Solarize', 0.18354873543483685, 84.1]
('BYOL', '256')   84.1
['ShearX', 0.0738254565954293, 'VerticalFlip', 0.1649378065796322, 'Rotate', -1.0, 84.23]
('SwaV', '256')   84.23


In [6]:
for exp_path, exp_name in zip(b32, b32_names):
    df, df_long, ops = get_data(exp_path)
    print(exp_name, " ", max(df_long['fitness']))

('SimSiam', '32')   84.08
('BYOL', '32')   84.12
('NNCLR', '32')   84.18
('SwaV', '32')   84.07


In [7]:
for exp_path, exp_name in zip(b256_svhn, b256_names_svhn):
    df, df_long, ops = get_data(exp_path)
    print(exp_name, " ", max(df_long['fitness']))

('NNCLR', '256')   93.09695759065765
('SwaV', '256')   93.20067609096495
('BYOL', '256')   92.67440073755378
('SimSiam', '256')   92.73970497848802


In [8]:
for exp_path, exp_name in zip(b32_svhn, b32_names_svhn):
    df, df_long, ops = get_data(exp_path)
    print(exp_name, " ", max(df_long['fitness']))

('SwaV', '32')   93.29287031346036
('NNCLR', '32')   92.76659496004918
('SimSiam', '32')   93.4081130915796
('BYOL', '32')   91.29917025199752


top 2 accuracies

In [9]:

def top2(model_path, dataset):
    backbone = largerCNN_backbone()
    model = finetune_model(backbone=backbone.backbone, in_features=backbone.in_features, num_outputs=dataset.num_classes)
    model.load_state_dict(torch.load(model_path))
    model = model.cuda()
    dataloader = DataLoader(dataset.test_data, batch_size=64,shuffle=False)
    total_correct = 0
    total = 0
    for X, y in dataloader:
        X, y = X.cuda(), y.cuda()
        outputs = model(X)
        vals, inds = torch.topk(outputs, k=2)
        output_ohe = torch.nn.functional.one_hot(inds, dataset.num_classes)
        output_ohe = torch.sum(output_ohe, dim=1)
        y_ohe = torch.nn.functional.one_hot(y, dataset.num_classes) 
        correct = torch.sum(output_ohe * y_ohe)
        total_correct += correct.item()
        total+= X.shape[0]
    return total_correct/total
    


In [10]:
for exp_path, exp_name in zip(b32, b32_names):
    models = glob.glob(os.path.join(os.path.dirname(exp_path), "models/*.pt"))
    if len(models) == 1:
        model_path = models[0]
    elif len(models) == 0:
        continue
    else:
        model_path = [m for m in models if "downstream" in m][0]
    top2_acc = top2(model_path, dataset=Cifar10())
    print(exp_name, " ", top2_acc)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('SimSiam', '32')   0.9258
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('BYOL', '32')   0.932
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('NNCLR', '32')   0.9281
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('SwaV', '32')   0.9302


In [11]:
for exp_path, exp_name in zip(b256, b256_names):
    models = glob.glob(os.path.join(os.path.dirname(exp_path), "models/*.pt"))
    if len(models) == 1:
        model_path = models[0]
    elif len(models) == 0:
        continue
    else:
        model_path = [m for m in models if "downstream" in m][0]
    top2_acc = top2(model_path, dataset=Cifar10())
    print(exp_name, " ", top2_acc)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('NNCLR', '256')   0.9253
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('SimSiam', '256')   0.9323
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('BYOL', '256')   0.9291
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
('SwaV', '256')   0.9219


In [12]:
for exp_path, exp_name in zip(b32_svhn, b32_names_svhn):
    models = glob.glob(os.path.join(os.path.dirname(exp_path), "models/*.pt"))
    if len(models) == 1:
        model_path = models[0]
    elif len(models) == 0:
        continue
    else:
        model_path = [m for m in models if "downstream" in m][0]
    top2_acc = top2(model_path, dataset=SVHN())
    print(exp_name, " ", top2_acc)

Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('SwaV', '32')   0.9657728948985863
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('NNCLR', '32')   0.9575522433927474
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('SimSiam', '32')   0.9582052858020897
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('BYOL', '32')   0.9612015980331899


In [13]:
for exp_path, exp_name in zip(b256_svhn, b256_names_svhn):
    models = glob.glob(os.path.join(os.path.dirname(exp_path), "models/*.pt"))
    if len(models) == 1:
        model_path = models[0]
    elif len(models) == 0:
        continue
    else:
        model_path = [m for m in models if "downstream" in m][0]
    top2_acc = top2(model_path, dataset=SVHN())
    print(exp_name, " ", top2_acc)

Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('NNCLR', '256')   0.9598570989551322
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('SwaV', '256')   0.9615857406269207
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('BYOL', '256')   0.9571681007990166
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train_32x32.mat
('SimSiam', '256')   0.965619237861094
